# 패키지 및 클래스 호출 

In [2]:
import geopandas as gpd
import pandas as pd
import re
import plotly.express as px
import os 

from shapely.wkt import loads
from glob import glob
from plotly.offline import plot
from shapely.geometry import Point


In [97]:
# 노션 지도 시각화 
class notion_map:
    def __init__(self,people,shp_path):
        self.shp = pd.concat([gpd.read_file(file,encoding = 'utf-8') for file in glob(shp_path+"/*.shp")]).reset_index(drop = True)
        self.people = pd.read_csv(people)
        
    def preprocessing(self):
        # 이부분도 수정하거나 없어질 가능성 있음
        self.people = self.people[self.people.columns[:3]].dropna()
        self.people.loc[self.people.시도명 == '양주시','시도명'] = '경기도'
        self.people.loc[self.people.시군구명 == '백석읍','시군구명'] = '양주시'
        
    def exp (self):
        megas = self.people.시도명.apply(lambda x: x.replace(" ",""))
        sggs= self.people.시군구명.apply(lambda x: x.replace(" ",''))
        centroid = [self.shp[((self.shp.MEGA_NM.apply(lambda x: re.search(mega,x)).isna()==False)&(self.shp.SIG_KOR_NM.apply(lambda x: re.search(sgg,x)).isna()==False))].centroid.tolist()[0] for mega,sgg in zip(megas,sggs)]
        ple['centroid'] = centroid
        
        return self.people
    
    def __del__(self):
        print("clear attribution")

class maps_visualization(notion_map):
    def __init__(self,people,shp_path):
        super().__init__(people,shp_path)
        super().preprocessing()
        super().exp()
        
    def visualization(self):
        gdf = gpd.GeoDataFrame(self.people,geometry = self.people.centroid,crs = 5179)
        # jitting 부분 (수정) <- 겹치는 경우가 2개이상일 경우는 고려되지 않았음 
        gdf['row_num'] = gdf.groupby(['시도명','시군구명']).cumcount() + 1
        gdf.geometry.x + 50
        gdf['x'] = gdf.geometry.x
        gdf['y'] = gdf.geometry.y
        gdf.loc[gdf.row_num == 2,'x'] = gdf.loc[gdf.row_num == 2,'x'] + 50
        gdf2 = gpd.GeoDataFrame(gdf,geometry = [Point(x,y) for x,y in zip(gdf.x,gdf.y)],crs = 5179)

        # 좌표변환 (UTM-k -> WGS84)
        gdf2 = gdf2.to_crs(4326)

        gdf2['lon'] = gdf2.geometry.x
        gdf2['lat'] = gdf2.geometry.y
        self.gdf = gdf2[['Name','시도명','시군구명','lon','lat']]
        return self.gdf
    
    def to_html(self,filename):        
        fig = px.scatter_mapbox(self.gdf,lat = 'lat',lon = 'lon',hover_name = 'Name',mapbox_style = 'open-street-map')
        plot(fig, filename=filename+'.html')
        return os.getcwd()+"/"+filename+'.html'

# 아래 과정(시도 정보 부여) 시군구 정보에 시도정보가 부여되어있다면 하지 않아도 무방합니다. 

### 전국의 시군구 정보 불러오기

In [8]:
shp2 = gpd.read_file('TL_SCCO_SIG.shp',encoding = 'cp949')

### 시도코드 정보 부여 & 시도명 부여 

In [16]:
shp2['MEGA_CD'] = shp2.SIG_CD.apply(lambda x: int(str(x)[:2]))

In [20]:
shp2 = shp2[shp2.MEGA_CD.isin([11,41,28])]

In [27]:
shp2.loc[shp2.MEGA_CD == 41,'MEGA_NM'] = '경기도'
shp2.loc[shp2.MEGA_CD == 11,'MEGA_NM'] = '서울시'
shp2.loc[shp2.MEGA_CD == 28,'MEGA_NM'] = '인천시'

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

###  .shp로 저장 

In [29]:
shp2.to_file("SGG_ctr.shp",encoding= 'utf-8')

# 아래의 사항을 주의해주세요    
> 1. shp.파일의 컬럼명-> 시도명은 MEGA_NM, 시군구명은 SIG_KOR_NM 혹시 다르다면, 코드 혹은 데이터 둘 중의 하나를 수정해주세요 
> 2. notion_maps는 maps_visualization에 상속되어 있어 굳이 실행시키지 않으셔도 됩니다.  
> 3. 시각화에 대한 옵션수정은 class에서 직접 수정해주셔야합니다. 
> 4. 기본적으로 하나의 과정이 끝나면 return하도록 되어있습니다. (혹시 return 되지않으신다면 해당 클래스의 속성을 직접 불러와주세요!)

### 초기 정보 입력 

In [52]:
shp_path = '/home/suyo1207/2021/notion/사내노션업데이트/' # shp파일 경로 (분할되어 있는 경우가 있어 경로로 입력하게 설정되어 있습니다. )

csv_file = "당신의 사는곳을 알려주세요 9572172efb494f81a1243b25bad749d8.csv" # csv파일명

### 클래스 실행 

In [101]:
maps = maps_visualization(csv_file,shp_path)

maps.visualization()

maps2.to_html_file('sample')